In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
from keras.utils import Sequence, to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

import numpy as np
import math

Using TensorFlow backend.


In [0]:
batch_size = 8

In [5]:
data = open('/content/gdrive/My Drive/Word learn/test.txt', 'r', encoding = 'utf-8').read()
#data = open('/content/gdrive/My Drive/Word learn/input.txt', 'r', encoding = 'utf-8').read()
data = data.lower().split('\n')
print(len(data))
print(len(data) % batch_size)


172
4


In [0]:
words = [x.split(' ') for x in data]
text = [x[i] for x in words for i in range(len(x))]
words = list( dict.fromkeys(text) )

In [0]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data)

In [8]:
total_words = len(tokenizer.word_index) + 1
print(total_words)

34


In [0]:
input_sentences = []
for line in data:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    tokenized_pred = token_list[:i+1]
    input_sentences.append(tokenized_pred)
    
lens = [len(x) for x in input_sentences]
max_seq_len = max(lens)
input_sentences = np.array(pad_sequences(input_sentences,
                                         maxlen = max_seq_len, padding = 'pre'))

In [0]:
print(input_sentences)

In [0]:
predictors, labels = input_sentences[:,:-1], input_sentences[:,-1]

In [11]:
print(predictors.shape)
print(labels.shape)
print(predictors.dtype)
print(labels.dtype)

(703, 5)
(703,)
int32
int32


In [0]:
class MyGenerator(Sequence):
  
  def __init__(self, predictors, labels, batch_size, n_classes):
      self.predictors, self.labels = predictors, labels
      self.batch_size = batch_size
      self.n_classes = n_classes
      self.x_dim = predictors.shape
      self.y_dim = labels.shape
      
  def __len__(self):
      return int(math.ceil(len(self.predictors) / self.batch_size))
    
  def __getitem__(self, index):
      batch_predictors = self.predictors[index * self.batch_size : (index + 1) * self.batch_size]
      batch_labels = self.labels[index * self.batch_size : (index + 1) * self.batch_size]
      
      #batch_predictors = np.reshape(batch_predictors, (batch_predictors.shape[0], 1, batch_predictors.shape[1]))
      #batch_labels = np.reshape(batch_labels, (batch_labels.shape[0], 1, batch_predictors.shape[1]))
      #print("PRED:", batch_predictors, "\n\n\n")
      #print("LAB:", batch_labels)
      return np.array(batch_predictors), to_categorical(batch_labels, num_classes = self.n_classes)

In [0]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout

In [0]:
generator = MyGenerator(predictors, labels, batch_size, total_words)

In [0]:
model = Sequential()
model.add(Embedding(total_words+1, 64))
#model.add(LSTM(500, return_sequences = True))

model.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [31]:
for i in range(0, predictors.shape[0], batch_size):
    x = predictors[i : i+batch_size]
    y = labels[i : i + batch_size]
    y = to_categorical(y, num_classes = total_words)

    print("Num of iteration:", i)
    print("X: ", x.shape)
    print("Y: ", y.shape)
    
    try:
        model.train_on_batch(x, y)
    except ValueError:
        y = np.reshape(y, (y.shape[0], 1, y.shape[1]))
        model.train_on_batch(x,y)


Num of iteration: 0
X:  (8, 5)
Y:  (8, 34)


ValueError: ignored